<a href="https://colab.research.google.com/github/amirkasaei/Modified-Human-Sperm-Morphology-Analysis/blob/main/Modified_Human_Sperm_Morphology_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

## **Torch**

In [ ]:
!pip install -q torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
pip install -q --upgrade torch

In [ ]:
pip install -q --upgrade torch torchvision

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
import torcheval
from torcheval.metrics.functional import binary_accuracy
from torchvision import transforms
import torchvision.transforms as T
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Resize

# PyTorch TensorBoard support
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

## **Others**

In [ ]:
!pip install --upgrade -q pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from sklearn.model_selection import KFold

import copy
import imageio
import PIL.Image as Image

import os
import sys
import time
import random
from tqdm import trange

from google.colab import drive

In [ ]:
# !pip install numba

# from numba import cuda
# device = cuda.get_current_device()
# device.reset()


In [ ]:
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/AI Projects/Meta-learning vision project/mhsma-dataset sperm images/mhsma/"

# **Preprocessing**

## **Load Data**

In [ ]:
!git clone https://github.com/amirkasaei/Modified-Human-Sperm-Morphology-Analysis/
path="/content/Modified-Human-Sperm-Morphology-Analysis/mhsma-dataset/mhsma/"

In [ ]:
size = '128'
x_train = np.load(path + 'x_' + size + '_train.npy').astype('float32')
y_acrosome_train = np.load(path + 'y_acrosome_train.npy').astype('float32')
y_head_train = np.load(path + 'y_head_train.npy').astype('float32')
y_tail_train = np.load(path + 'y_tail_train.npy').astype('float32')
y_vacuole_train = np.load(path + 'y_vacuole_train.npy').astype('float32')

x_valid = np.load(path+'x_' + size + '_valid.npy').astype('float32')
y_acrosome_valid = np.load(path + 'y_acrosome_valid.npy').astype('float32')
y_head_valid = np.load(path + 'y_head_valid.npy').astype('float32')
y_tail_valid = np.load(path + 'y_tail_valid.npy').astype('float32')
y_vacuole_valid = np.load(path + 'y_vacuole_valid.npy').astype('float32')

x_test = np.load(path+'x_' + size + '_test.npy').astype('float32')
y_acrosome_test = np.load(path + 'y_acrosome_test.npy').astype('float32')
y_head_test = np.load(path + 'y_head_test.npy').astype('float32')
y_tail_test = np.load(path + 'y_tail_test.npy').astype('float32')
y_vacuole_test = np.load(path + 'y_vacuole_test.npy').astype('float32')

print('x_train shape:', x_train.shape, '----- y_acrosome_train shape:', y_acrosome_train.shape)
print('x_valid shape:', x_valid.shape, '----- y_acrosome_valid shape:', y_acrosome_valid.shape)
print('x_test shape:', x_test.shape, '----- y_acrosome_test shape:', y_acrosome_test.shape)

In [ ]:
np.count_nonzero(y_head_train == 0)/y_head_train.shape[0]

In [ ]:
np.count_nonzero(y_head_test == 0)/y_head_test.shape[0]

## **Reshape Data**

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
# y_acrosome_train = y_acrosome_train.reshape((-1,1))
# y_head_train = y_head_train.reshape((-1,1))
# y_tail_train = y_tail_train.reshape((-1,1))
# y_vacuole_train = y_vacuole_train.reshape((-1,1))

x_valid = x_valid.reshape((x_valid.shape[0], x_valid.shape[1], x_valid.shape[2], 1))
# y_acrosome_valid = y_acrosome_valid.reshape((-1,1))
# y_head_valid = y_head_valid.reshape((-1,1))
# y_tail_valid = y_tail_valid.reshape((-1,1))
# y_vacuole_valid = y_vacuole_valid.reshape((-1,1))

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))
# y_acrosome_test = y_acrosome_test.reshape((-1,1))
# y_head_test = y_head_test.reshape((-1,1))
# y_tail_test = y_tail_test.reshape((-1,1))
# y_vacuole_test = y_vacuole_test.reshape((-1,1))

In [ ]:
input_shape = x_train.shape[1:]
input_shape

## **Torch**

In [ ]:
y_acrosome_train_tensor = torch.from_numpy(y_acrosome_train)
y_head_train_tensor = torch.from_numpy(y_head_train)
y_tail_train_tensor = torch.from_numpy(y_tail_train)
y_vacuole_train_tensor = torch.from_numpy(y_vacuole_train)

y_acrosome_valid_tensor = torch.from_numpy(y_acrosome_valid)
y_head_valid_tensor = torch.from_numpy(y_head_valid)
y_tail_valid_tensor = torch.from_numpy(y_tail_valid)
y_vacuole_valid_tensor = torch.from_numpy(y_vacuole_valid)

y_acrosome_test_tensor = torch.from_numpy(y_acrosome_test)
y_head_test_tensor = torch.from_numpy(y_head_test)
y_tail_test_tensor = torch.from_numpy(y_tail_test)
y_vacuole_test_tensor = torch.from_numpy(y_vacuole_test)


In [ ]:
Tensor = T.Compose([
    ToTensor()
])

PILimage = T.Compose([
    ToPILImage()
])

Tensor224 = T.Compose([
    ToTensor(),
    Resize(size = (224,224), interpolation=torchvision.transforms.InterpolationMode.NEAREST_EXACT)
])

In [ ]:
x_train_tensor = torch.stack([Tensor(img) for img in x_train])

x_valid_tensor = torch.stack([Tensor(img) for img in x_valid])

x_test_tensor = torch.stack([Tensor(img) for img in x_test])

In [ ]:
x_train_tensor.shape

In [ ]:
plt.imshow(PILimage(x_train_tensor[0]))

## **Data Normalization**

### **Mean**

In [ ]:
mean_train = x_train_tensor.view(1, -1).mean(dim=1)
mean_valid = x_valid_tensor.view(1, -1).mean(dim=1)
mean_test = x_test_tensor.view(1, -1).mean(dim=1)

In [ ]:
mean_train, mean_valid, mean_test

### **Standard Deviation**

In [ ]:
std_train = x_train_tensor.view(1, -1).std(dim=1)
std_valid = x_valid_tensor.view(1, -1).std(dim=1)
std_test = x_test_tensor.view(1, -1).std(dim=1)

In [ ]:
std_train, std_valid, std_test

### **Normalizarion**

In [ ]:
Normalizer = Normalize(128, 8)
x_train_norm = Normalizer(x_train_tensor)

In [ ]:
Normalizer = Normalize(mean_valid, std_valid)
x_valid_norm = Normalizer(x_valid_tensor)

In [ ]:
Normalizer = Normalize(mean_test, std_test)
x_test_norm = Normalizer(x_test_tensor)

In [ ]:
plt.imshow(PILimage(x_train_norm[0]))

## **Data Augmentation**

## **Equal Distribution**

In [ ]:
classes, class_samples = np.unique(y_head_train_tensor, return_counts=True)
classes, class_samples

In [ ]:
false_ids = np.random.choice(np.where(y_head_train_tensor == classes[0])[0], size=class_samples[1])
true_ids = np.where(y_head_train_tensor == classes[1])[0]
ids = np.sort(np.concatenate((false_ids, true_ids), axis=0))

In [ ]:
x_train_equal = x_train_tensor[ids]
y_head_train_equal = y_head_train_tensor[ids]
x_train_equal.shape, y_head_train_equal.shape

In [ ]:
y_head_train_equal

## **RGB**

In [ ]:
x_train_rgb = torch.stack([img.repeat(3, 1, 1) for img in x_train_tensor])
# x_train_rgb = torch.stack([img.repeat(3, 1, 1) for img in x_train_equal])

x_valid_rgb = torch.stack([img.repeat(3, 1, 1) for img in x_valid_tensor])

x_test_rgb = torch.stack([img.repeat(3, 1, 1) for img in x_test_tensor])

In [ ]:
plt.imshow(PILimage(x_train_rgb[0]))

## **Data Loader**

### **Dataset Class**

In [ ]:
class MHSMADataset(Dataset):
  def __init__(self, x, y, transform=None, target_transform=None):
    self.img_labels = y
    self.imgs = x
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    image = self.imgs[idx]
    label = self.img_labels[idx]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

### **Default Dataloader**

In [ ]:
train_data = MHSMADataset(x=x_train_tensor, y=y_head_train_tensor)

valid_data = MHSMADataset(x=x_valid_tensor, y=y_head_valid_tensor)

test_data = MHSMADataset(x=x_test_tensor, y=y_head_test_tensor)

In [ ]:
dataset_sizes = {}

dataset_sizes['train'] = len(train_data)
dataset_sizes['valid'] = len(valid_data)
dataset_sizes['test'] = len(test_data)

In [ ]:
batch_size=64

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
dataloaders = {}

dataloaders['train'] = train_dataloader
dataloaders['valid'] = valid_dataloader
dataloaders['test'] = test_dataloader

In [ ]:
dataloader_sizes = {}

dataloader_sizes['train'] = len(train_dataloader)
dataloader_sizes['valid'] = len(valid_dataloader)
dataloader_sizes['test'] = len(test_dataloader)

### **RGB Dataloader**

In [ ]:
train_data_rgb = MHSMADataset(x=x_train_rgb, y=y_head_train_tensor)
# train_data_rgb = MHSMADataset(x=x_train_rgb, y=y_head_train_equal)

valid_data_rgb = MHSMADataset(x=x_valid_rgb, y=y_head_valid_tensor)

test_data_rgb = MHSMADataset(x=x_test_rgb, y=y_head_test_tensor)

In [ ]:
batch_size=64

train_dataloader_rgb = DataLoader(train_data_rgb, batch_size=batch_size, shuffle=True, num_workers=2)
valid_dataloader_rgb = DataLoader(valid_data_rgb, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader_rgb = DataLoader(test_data_rgb, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
dataloaders_rgb = {}

dataloaders_rgb['train'] = train_dataloader_rgb
dataloaders_rgb['valid'] = valid_dataloader_rgb
dataloaders_rgb['test'] = test_dataloader_rgb

In [ ]:
dataloader_rgb_sizes = {}

dataloader_rgb_sizes['train'] = len(train_dataloader_rgb)
dataloader_rgb_sizes['valid'] = len(valid_dataloader_rgb)
dataloader_rgb_sizes['test'] = len(test_dataloader_rgb)

In [ ]:
train_batch_num = len(train_dataloader)

In [ ]:
# train_features, train_labels = next(iter(train_dataloader))

# **Models**

## **Helper Functions**

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

### **Train**

In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloader, dataloader_size, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 20)

    # Each epoch has a training and validation phase
    for phase in ['train', 'valid']:
      if phase == 'train':
        model.train()  # Set model to training mode
      else:
        model.eval()   # Set model to evaluate mode

      running_loss = 0.0
      running_corrects = 0
      # running_acc = 0.0

      # Iterate over data.
      for inputs, labels in dataloader[phase]:
        inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)
          # acc = binary_accuracy(outputs.reshape(outputs.shape[0]), labels.reshape(labels.shape[0]), threshold=0.5)

          # backward + optimize only if in training phase
          if phase == 'train':
            loss.backward()
            optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        # running_acc += acc

        # if phase == 'train':
        #   scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

      # deep copy the model
      if phase == 'valid' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    print()

  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Best val Acc: {best_acc:4f}')

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

### **Evaluation**

In [ ]:
def evaluate_model(model, criterion, dataloader, dataloader_size, dataset_size):
  since = time.time()

  print('-' * 20)

  # Each epoch has a training and validation phase
  model.eval()   # Set model to evaluate mode

  running_loss = 0.0
  running_corrects = 0
  # running_acc = 0.0

  # Iterate over data.
  for inputs, labels in dataloader:
    inputs = inputs.to(device)
    labels = labels.type(torch.LongTensor).to(device)

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

  eval_loss = running_loss / dataset_size
  eval_acc = running_corrects.double() / dataset_size

  print(f'test Loss: {eval_loss:.4f} Acc: {eval_acc:.4f}')

## **VGG**

### **Configuration**

In [ ]:
vgg = torchvision.models.vgg19(pretrained=True)
vgg.classifier

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
# for param in vgg.features.parameters():
#     param.require_grad = False

In [ ]:
num_ftrs = vgg.classifier[0].in_features
vgg.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 2048),
    nn.ReLU(),
    nn.Dropout(0.9),
    nn.Linear(2048, 1024),
    nn.Sigmoid(),
    nn.Dropout(0.9),
    nn.Linear(1024, 2)
)

vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.9, inplace=False)
  (3): Linear(in_features=2048, out_features=1024, bias=True)
  (4): Sigmoid()
  (5): Dropout(p=0.9, inplace=False)
  (6): Linear(in_features=1024, out_features=2, bias=True)
)

### **Train**

In [ ]:
vgg = vgg.to(device)

learning_rate = 3e-3
epochs = 100

# criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
# accuracy = metrics.accuracy_score()

# Observe that all parameters are being optimized
# optimizer = torch.optim.Adam(vgg.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(vgg.parameters(), lr=learning_rate, momentum=0.9)
optimizer = torch.optim.Adadelta(vgg.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
vgg = train_model(vgg, criterion,optimizer, exp_lr_scheduler, dataloaders_rgb, dataloader_rgb_sizes, num_epochs=epochs)

Epoch 1/100
--------------------
train Loss: 0.5263 Acc: 0.2680
valid Loss: 0.7123 Acc: 0.3167

Epoch 2/100
--------------------
train Loss: 0.5792 Acc: 0.2590
valid Loss: 0.6885 Acc: 0.5125

Epoch 3/100
--------------------
train Loss: 0.5409 Acc: 0.2570
valid Loss: 0.6838 Acc: 0.5958

Epoch 4/100
--------------------
train Loss: 0.4911 Acc: 0.2910
valid Loss: 0.6623 Acc: 0.7333

Epoch 5/100
--------------------
train Loss: 0.5337 Acc: 0.2700
valid Loss: 0.6735 Acc: 0.6917

Epoch 6/100
--------------------
train Loss: 0.5028 Acc: 0.2730
valid Loss: 0.6586 Acc: 0.7333

Epoch 7/100
--------------------
train Loss: 0.5117 Acc: 0.2720
valid Loss: 0.6630 Acc: 0.7333

Epoch 8/100
--------------------
train Loss: 0.5261 Acc: 0.2630
valid Loss: 0.6649 Acc: 0.7250

Epoch 9/100
--------------------
train Loss: 0.5087 Acc: 0.2690
valid Loss: 0.6548 Acc: 0.7250

Epoch 10/100
--------------------
train Loss: 0.4720 Acc: 0.3050
valid Loss: 0.6404 Acc: 0.7292

Epoch 11/100
--------------------
train

### **Evaluarte**

In [ ]:
evaluate_model(vgg, criterion, dataloaders_rgb, dataloader_rgb_sizes)

--------------------
test Loss: 0.6116 Acc: 0.7300


## **ViT**

### **Configuration**

In [ ]:
vit = torchvision.models.vit_b_16(pretrained=True)

In [ ]:
vit.heads

Sequential(
  (head): Linear(in_features=768, out_features=1000, bias=True)
)

In [ ]:
num_ftrs = vit.heads.head.in_features
vit.heads =  nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.9),
    nn.Linear(512, 256),
    nn.Sigmoid(),
    nn.Dropout(0.9),
    nn.Linear(256, 2))

In [ ]:
vit.heads

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.9, inplace=False)
  (3): Linear(in_features=512, out_features=256, bias=True)
  (4): Sigmoid()
  (5): Dropout(p=0.9, inplace=False)
  (6): Linear(in_features=256, out_features=2, bias=True)
)

### **Train**

In [ ]:
vit = vit.to(device)

learning_rate = 3e-3
epochs = 100

criterion = nn.CrossEntropyLoss()
# accuracy = metrics.accuracy_score()

# Observe that all parameters are being optimized
# optimizer = torch.optim.Adam(vgg.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(vgg.parameters(), lr=learning_rate, momentum=0.9)
optimizer = torch.optim.Adadelta(vit.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
vit = train_model(vit, criterion, optimizer, exp_lr_scheduler, dataloaders_rgb, dataloader_rgb_sizes, num_epochs=epochs)

### **Evaluarte**

In [ ]:
evaluate_model(vit, criterion, dataloaders_rgb, dataloader_rgb_sizes)

--------------------
test Loss: 0.6116 Acc: 0.7300


## **CNN**

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3)
    self.pool = nn.MaxPool2d(2, 2)
    self.flatten = nn.Flatten()
    self.linrear1 = nn.Linear(in_features=16 * 30 * 30, out_features=128)
    self.linrear2 = nn.Linear(in_features=128, out_features=32)
    self.output = nn.Linear(in_features=32, out_features=1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.pool(x)
    x = self.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(-1, 16 * 30 * 30)
    x = self.flatten(x)
    x = self.relu(self.linrear1(x))
    x = self.relu(self.linrear2(x))
    x = self.sigmoid(self.output(x))
    return x


In [ ]:
model = CNN()

In [ ]:
model = model.to(device)

In [ ]:
learning_rate = 1e-3
epochs = 100

criterion = nn.BCELoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, dataloaders, dataloader_sizes, num_epochs=epochs)